# Dowloading and preparing the Dataset



In [8]:
#uploading kaggle.json file to download data
from google.colab import files 
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"amitpant7","key":"c050fe3c1a4bd4b58a81f484188d3eaf"}'}

In [9]:
!mv kaggle.json /root/.kaggle

In [10]:
import kaggle 
!kaggle datasets download -d salader/dogs-vs-cats

dogs-vs-cats.zip: Skipping, found more recently modified local copy (use --force to force download)


In [11]:
# !unzip dogs-vs-cats
!unzip dogs-vs-cats

Archive:  dogs-vs-cats.zip
replace dogs_vs_cats/test/cats/cat.10.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: N


#The Model Part

In [12]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.activations import relu, linear
from tensorflow.keras.applications import ResNet50

In [13]:
# #Preparing Data
# from keras.preprocessing.image import ImageDataGenerator
# train_datagen = ImageDataGenerator(rescale=1./255)
# train_data_gen =  train_datagen.flow_from_directory(
#     './PetImages',
#     target_size=(150, 150),
#     batch_size=32,
#     class_mode='binary')

In [14]:
#preparing the base convolution
conv_base = ResNet50(weights='imagenet',
                     include_top = False,
                     input_shape= (224,224,3))

In [15]:
conv_base.summary()

Model: "resnet50"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 230, 230, 3)  0           ['input_2[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 112, 112, 64  9472        ['conv1_pad[0][0]']              
                                )                                                                 
                                                                                           

In [16]:
#Preparing the model
model = Sequential() 
model.add(conv_base)
model.add(Flatten())
model.add(Dense(256, activation= 'relu'))
model.add(Dense(1, activation= 'sigmoid'))

In [17]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50 (Functional)       (None, 7, 7, 2048)        23587712  
                                                                 
 flatten_1 (Flatten)         (None, 100352)            0         
                                                                 
 dense_2 (Dense)             (None, 256)               25690368  
                                                                 
 dense_3 (Dense)             (None, 1)                 257       
                                                                 
Total params: 49,278,337
Trainable params: 49,225,217
Non-trainable params: 53,120
_________________________________________________________________


In [18]:
#Freezing the convolutional Layers
conv_base.trainable = False

In [19]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50 (Functional)       (None, 7, 7, 2048)        23587712  
                                                                 
 flatten_1 (Flatten)         (None, 100352)            0         
                                                                 
 dense_2 (Dense)             (None, 256)               25690368  
                                                                 
 dense_3 (Dense)             (None, 1)                 257       
                                                                 
Total params: 49,278,337
Trainable params: 25,690,625
Non-trainable params: 23,587,712
_________________________________________________________________


#Preparing the Dataset

In [20]:
#Generators 
train_dataset = tf.keras.utils.image_dataset_from_directory(
    './dogs_vs_cats/train',
    labels= 'inferred',
    label_mode = 'int',
    batch_size = 32,
    image_size = (224, 224)
)

val_dataset = tf.keras.utils.image_dataset_from_directory(
    './dogs_vs_cats/test',
    labels= 'inferred',
    label_mode = 'int',
    batch_size = 32,
    image_size = (224, 224)
)


Found 20000 files belonging to 2 classes.
Found 5000 files belonging to 2 classes.


In [21]:
#Normalize 
def process(image, label):
  # channels = tf.shape(image)[-1]
  # # if channels < 3:
  # #    image = tf.image.grayscale_to_rgb(image)

  image = tf.cast(image/255., tf.float32)
  return image, label

train_set = train_dataset.map(process)
val_set = val_dataset.map(process)

In [22]:
#compiling the model
model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

In [ ]:
#Training the Data 
history = model.fit(train_set, epochs =5 , validation_data = val_set)

Epoch 1/5
625/625 [==============================] - 4009s 6s/step - loss: 0.9356 - accuracy: 0.6146 - val_loss: 0.6155 - val_accuracy: 0.6454
Epoch 2/5
625/625 [==============================] - 4062s 7s/step - loss: 0.5970 - accuracy: 0.6793 - val_loss: 0.6403 - val_accuracy: 0.6114
Epoch 3/5
625/625 [==============================] - 4065s 7s/step - loss: 0.5788 - accuracy: 0.6912 - val_loss: 0.5730 - val_accuracy: 0.7072
Epoch 4/5
541/625 [========================>.....] - ETA: 7:21 - loss: 0.5650 - accuracy: 0.7058

#Ploting the performance of the Mode

In [ ]:
#Plotting Accuracy
import matplotlib.pyplot as plt 
plt.plot(history.history['accuracy'], color= 'red', label = 'train')
plt.plot(history.history['val_accuray'], color = 'blue', label = 'Validation')
plt.legend()
plt.show()

In [ ]:
#Plotting Loss
plt.plot(history.history['loss'], color= 'red', label = 'train')
plt.plot(history.history['val_loss'], color = 'blue', label = 'train')
plt.legend()
plt.show()